# **Agentic AI - NBA Stats**
## **Author:** Tanner Rapp
### **Data Source:** Walsh, W. (n.d.). NBA Database [Data set]. Kaggle. https://www.kaggle.com/datasets/wyattowalsh/basketball

#### About this Program
---

This program automates NBA data analysis using an “agentic” (autonomous) AI workflow. It connects to structured datasets—like player stats, draft history, or team performance—by uploading CSV files and allowing a GPT model (e.g., GPT-4.1-mini) with a code interpreter to read, query, and reason over them. The script first identifies which tables are relevant to the user’s question (for example, selecting draft_history when asked about Lakers’ recent draft picks), then executes Python code inside the model’s sandbox to explore and return structured outputs and analytical notes. Essentially, it’s a dynamic data-assistant pipeline that lets a model act as both analyst and coder for NBA-related data exploration

#### Import Libraries
---

In [1]:
import kagglehub
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import os
import json
import glob
from dotenv import load_dotenv
from openai import OpenAI

#### Download Dataset
---

In [3]:
sample_file = os.path.join(os.getcwd(),"csv", "game.csv")

if not os.path.exists(sample_file):
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files('wyattowalsh/basketball', path='.', unzip=True)
else:
    print("Dataset already exists")

Dataset already exists


In [4]:
# List of available data from the Kaggle source
tables = [
    "common_player_info",
    "draft_combine_stats",
    "draft_history",
    "game",
    "game_info",
    "game_summary",
    "inactive_players",
    "line_score",
    "officials",
    "other_stats",
    "play_by_play",
    "player",
    "team",
    "team_details",
    "team_history",
    "team_info_common"
]

In [5]:
#Loading in the data
dfs = {}
for name in tables:
    file_path = os.path.join(os.getcwd(), "csv", f"{name}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        dfs[name] = df
        print(f"✅ Loaded {name}.csv — shape: {df.shape}")
    else:
        print(f"⚠️ File not found: {name}.csv")


✅ Loaded common_player_info.csv — shape: (4171, 33)
✅ Loaded draft_combine_stats.csv — shape: (1202, 47)
✅ Loaded draft_history.csv — shape: (7990, 14)
✅ Loaded game.csv — shape: (65698, 55)
✅ Loaded game_info.csv — shape: (58053, 4)
✅ Loaded game_summary.csv — shape: (58110, 14)
✅ Loaded inactive_players.csv — shape: (110191, 9)
✅ Loaded line_score.csv — shape: (58053, 43)
✅ Loaded officials.csv — shape: (70971, 5)
✅ Loaded other_stats.csv — shape: (28271, 26)
✅ Loaded play_by_play.csv — shape: (13592899, 34)
✅ Loaded player.csv — shape: (4831, 5)
✅ Loaded team.csv — shape: (30, 7)
✅ Loaded team_details.csv — shape: (25, 14)
✅ Loaded team_history.csv — shape: (52, 5)
✅ Loaded team_info_common.csv — shape: (0, 26)


#### Upload Files to OpeanAI Assistants 
---

In [8]:
# Open OpenAI Client through an API key
load_dotenv(".env")
client = OpenAI()

In [9]:
DATA_DIR = os.path.join(os.getcwd(), "csv")

# Collect all CSVs you want the analyst to see
csv_paths = glob.glob(os.path.join(DATA_DIR, "*.csv"))
assert csv_paths, f"No CSVs found in {DATA_DIR}"

In [10]:
# Upload them to OpenAI Files (purpose='assistants')
uploaded_files = []
for p in csv_paths:
    if os.path.basename(p) == "play_by_play.csv":
        continue # Have to skip this dataset because it is way too big to be uploaded (exceeded openai's limit)
        
    f = client.files.create(file=open(p, "rb"), purpose="assistants")
    uploaded_files.append({"name": os.path.basename(p), "id": f.id})


In [11]:
user_content=[]
for f in uploaded_files:
    user_content.append({"name": f['name'], "file_id": f["id"]})

#### Config You Control
---

In [12]:
USER_QUERY = "Who did the lakers draft most recentl"

#### Define AI Agent Roles
---

In [13]:
IDENTIFIER_SYS = (
        "You read the user's query and decide which tables are needed. "
        "Prefer the smallest set of tables that can fully answer the question. "
        "For player game totals by game, favor 'game', 'game_summary', and 'player' (for IDs). "
        "Return your decision by calling select_tables()."
)
ANALYST_SYS = (
        "You have Python via the code_interpreter tool. "
        "Goal: given a user query and a list of relevant tables, load the CSVs from DATA_DIR, "
        "locate the player (e.g., LeBron James), identify the latest full season ('last season'), "
        "and return a concise table of game-level totals (one row per game) including date, opponent, "
        "and key box score totals (points, rebounds, assists, etc.). Print the final dataframe head(10). "
        "If a needed file is missing, clearly state which is missing."
)
WRITER_SYS =  ( 
        "Turn analytic output into a short, clear paragraph. "
        "Summarize what was computed and highlight key takeaways. Keep it factual and concise."
)

#### Run Agent 1: Table Identifier
---

In [29]:
id_resp = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "code_interpreter",
           "container": { "type": "auto", "file_ids": [f["file_id"] for f in user_content] }}],
    input=[
        {"role":"system","content":IDENTIFIER_SYS},
        {"role":"user","content":
         f"USER QUERY: {USER_QUERY}\n"
         "\n\nTABLE_DESCRIPTIONS (JSON):\n"+json.dumps(table_descriptions, indent=2)+
         "\n\nYou can use python to look through each dataset from file_ids in user content for actual columns to identify what is needed to answer the user query."
         "n\n\Confirm if you can access the files through python with a simple statement saying that you can or can't"
         "n\n\Keep is super short to which tables you will use and why in a bullet point format. Nothing else except if you can read the python files"
         "n\n\Keep it to under 30 words."
        }
    ],
    include=["code_interpreter_call.outputs"] 
)

print("\n--- ANALYST OUTPUT ---\n", id_resp.output_text)



--- ANALYST OUTPUT ---
 I can access the files through python.

- Use draft_history to find the most recent Lakers draft picks.
- Use team to confirm Lakers team ID if needed for filtering.


#### Run Agent 2: Data Analyst
---

In [31]:
analyst_resp = client.responses.create(
    model="gpt-4.1",
    tools=[{"type": "code_interpreter",
           "container": { "type": "auto", "file_ids": [f["file_id"] for f in user_content] }}],
    input=[
        {"role": "system", "content": [{"type": "input_text", "text": ANALYST_SYS}]},
        {"role": "user", "content": [
            {"type": "input_text", "text":
                "Here is the task and context:\n"
                f"- User query: {USER_QUERY}\n"
                "INSTRUCTIONS:\n"
                "1) Use Python to read/analyze necessary CSVs from the attached files.\n"
                f"2) Identify relevant tables and columns from {id_resp.output_text}.\n"
                "3) Resolve player/team IDs if needed.\n"
                "4) Filter/join/aggregate as required.\n"
                "5) Show a concise DataFrame and head(10).\n"
                "6) If a file/column is missing, state which and continue."
                "7) Regardless of whether or not you have all you need, just compute what you can with what is given from chosen_tables. You must return some stats"
            }
        ]}
    ],
    include=["code_interpreter_call.outputs"] 
)

print("\n--- ANALYST OUTPUT ---\n", analyst_resp.output_text)




--- ANALYST OUTPUT ---
 Let's follow your instructions step-by-step:

### 1. Relevant files:
- `draft_history.csv` — needed to find recent Lakers draft picks.
- `team.csv` — useful to determine the Lakers' team ID.
- (We don't need other tables for just the most recent draft pick.)

Let's load these files and proceed with the task: 
- Find the Lakers' team ID from `team.csv`.
- Use this team ID to filter `draft_history.csv` for Lakers picks.
- Find the most recent draft pick(s).
- Output a concise DataFrame (top 10 if more than one).

Let me start by loading the relevant files and inspecting them.- The `team.csv` file uses `id` for team ID and `full_name` for team name.
- The `draft_history.csv` uses `team_id` for which team made the pick; `player_name`, `season`, etc. are for the draft pick.

Let's:
1. Find the Lakers' team ID (`full_name` contains 'Lakers').
2. Filter `draft_history.csv` rows where `team_id` is the Lakers.
3. Find the latest (`season` max) pick(s).
4. Display a conc

#### Run Agent 3: Writer
---

In [33]:
writer_resp = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {"role":"system","content":WRITER_SYS},
        {"role":"user","content":"Turn this into a short summary:\n\n"+analyst_resp.output_text+"\n\n"+
        "You must show any tables from the analyst_resp.output_text to go along with your response"}
    ]
)
print("\n--- WRITER OUTPUT ---\n", writer_resp.output_text)


--- WRITER OUTPUT ---
 The analysis identified the Los Angeles Lakers' team ID from the `team.csv` file and used it to filter the `draft_history.csv` data for the most recent draft picks. The latest season for draft picks is 2023, and the most recent Lakers draft picks are Jalen Hood-Schifino (1st round, 17th overall) and Mojave King (2nd round, 47th overall). These players represent the Lakers' newest additions based on the available draft history.

|      player_name        | season | round_number | round_pick | overall_pick |   organization        |
|------------------------|--------|--------------|------------|--------------|-----------------------|
| Jalen Hood-Schifino    | 2023   |      1       |     17     |      17      | Indiana               |
| Mojave King            | 2023   |      2       |     17     |      47      | Ignite (G League)     |
